 # Notebook 02 — Authentication & Data Loading



 Authenticates with Google, mounts Drive, and loads all data sources into the pipeline object so that subsequent notebooks can start from a known, inspectable state.


 Workflow:

 1. Mount Drive and inject repo into `sys.path`

 2. Authenticate and create pipeline

 3. Load logger temperature data

 4. Load Curve6 / Curve7 microCT time axes

 5. Inspect a raw branch as a sanity check

 ## Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install dtaidistance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 41.0 MB/s eta 0:00:00


 ## Inject repo into Python path

In [3]:
import sys

# Edit REPO_PATH if you cloned the repo to a different location inside MyDrive
REPO_PATH = '/content/rhodopipeline'

if REPO_PATH not in sys.path:
    sys.path.insert(0, REPO_PATH)

import rhodopipeline
print('rhodopipeline imported successfully')


rhodopipeline imported successfully


 ## Create pipeline and authenticate

In [4]:
from rhodopipeline import RhodolithPipeline, CONFIG

pipeline = RhodolithPipeline(CONFIG)
pipeline.authenticate()


STEP 1: AUTHENTICATION
Mounted at /content/drive
✓ Authenticated.



 ## Load logger temperature data

In [5]:
pipeline.load_temperature_data()

# Inspect date range and basic statistics
print('\nFirst 5 days:')
print(pipeline.temp_full.head())
print(f'\nDate range: {pipeline.temp_full.index[0].date()} → {pipeline.temp_full.index[-1].date()}')
print(f'Min: {pipeline.temp_full.min():.2f} °C   Max: {pipeline.temp_full.max():.2f} °C')


STEP 2: LOAD TEMPERATURE
✓ Temp loaded: 130 days.
  Range: 26.07–32.72 °C


First 5 days:
Index
2024-03-16    26.663500
2024-03-17    26.704583
2024-03-18    26.268042
2024-03-19    26.316750
2024-03-20    26.276208
Freq: D, Name: temp.exposed.af, dtype: float64

Date range: 2024-03-16 → 2024-07-23
Min: 26.07 °C   Max: 32.72 °C


 ## Load Curve6 and Curve7 time axes

In [6]:
pipeline.load_curve6_curve7()

if pipeline.curve6 is not None:
    print('\nCurve6 head:')
    print(pipeline.curve6.head())

if pipeline.curve7 is not None:
    print('\nCurve7 head:')
    print(pipeline.curve7.head())


LOAD CURVE6 & CURVE7 (Rhodo25_data)
✓ Curve6 loaded (78 increments)
✓ Curve7 loaded (81 increments)


Curve6 head:
  Increment Start (um) End (um) Width (um) Mean Density Mean Growth Rate  \
0         1          0    21.27      21.27   39012.2893      16.84104418   
1         2      21.27    31.91      10.64  38324.03516      17.14385257   
2         3      31.91    77.13      45.22   37374.4105      17.57949829   
3         4      77.13   119.69      42.56  37964.12941      17.30605345   
4         5     119.69   130.33      10.64  37261.37511      17.63302866   

             DateTime  
0 2024-03-14 01:49:10  
1 2024-03-15 01:48:46  
2 2024-03-16 16:05:25  
3 2024-03-19 03:51:21  
4 2024-03-20 16:40:51  

Curve7 head:
  Increment Start (um) End (um) Width (um) Mean Density Mean Growth Rate  \
0         1          0    16.99      16.99  39042.68498      16.82788072   
1         2      16.99    48.14      31.15  38034.98489      17.27560509   
2         3      48.14     62.3      14.16

 ## Sanity check: inspect raw branch AFE5-1

In [7]:
df_branch1, name = pipeline.get_raw_branch_data(1)

if df_branch1 is not None:
    print(f'\nBranch loaded: {name}')
    print(f'Shape: {df_branch1.shape}')
    print('\nFirst 5 rows:')
    print(df_branch1.head())
    print('\nColumns:')
    print(list(df_branch1.columns))
else:
    print('Branch 1 not found — check that the Google Sheets workbook name is correct in CONFIG.')



Branch loaded: afe5-1
Shape: (43, 28)

First 5 rows:
  Positions     Sample            Li/Mg Li7/Ca43(mmol/mol)            B/Mg  \
0        50  AFE5-1-43  0.0005318077022           0.161928  0.002475131862   
1       100  AFE5-1-42  0.0006983981744           0.188527  0.002427651125   
2       150  AFE5-1-41  0.0006515578253           0.178362  0.002548678159   
3       200  AFE5-1-40  0.0006838058167           0.186214  0.002653077262   
4       250  AFE5-1-39  0.0007222975647           0.188009  0.002675688551   

  B11/Ca43(mmol/mol)         Na/Mg Na23/Ca43(mmol/mol) Mg25/Ca43(mmol/mol)  \
0           0.753643  0.1750402317             53.2973             304.486   
1           0.655325  0.2170884857             58.6013             269.942   
2           0.697693  0.2123142902             58.1204             273.747   
3           0.722486  0.1974052585             53.7574              272.32   
4           0.696463  0.2100060317             54.6631             260.293   

  Al27/C